### Mass segregation

Analysis of the mass segregation in the stellar clusters using different indices <br>

In [1]:
using PyCall
using Distributions
using Statistics

using Distances
using Random , Formatting
using AstroLib

rootdir = "/home/stephane/Science/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

import PyPlot

@pyimport astroquery.vizier as vizier
@pyimport astropy.units as u
@pyimport astropy.coordinates as coord


## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)